#通过本地database提取数据计算因子

#1. 5年营业收入成长率

In [84]:
import numpy as np
import pandas as pd
import tushare as ts
import security_fundamental_data

ts.set_token('e9628663b45f87ea92e39aaa7127063830ebd5090bc1e8943138b84f')
pro = ts.pro_api()

In [2]:
indicator = '每股营业收入'
target_period = '20220331'
threshold_date = '20220520'

period_list = security_fundamental_data.get_TTM_period_list(target_period)
df = security_fundamental_data.get_local_db_stock_fin_abstract(period=target_period, indicator=indicator)
df = df.loc[df['ann_date'] <= threshold_date].copy()
selected_sec_ls = df.code.tolist()
if len(period_list) > 1:
    for period in period_list[1:]:
        new_df = security_fundamental_data.get_local_db_stock_fin_abstract(period=period, indicator=indicator)
        new_df = new_df.loc[new_df['ann_date'] <= threshold_date].copy()
        selected_sec_ls = list(set(selected_sec_ls) & set(new_df.code.tolist()))
        df = pd.concat([df, new_df], ignore_index=True)
df_filtered = df.loc[df['code'].isin(selected_sec_ls)]  # 确保数据完整获取
# 得到指定指标的TTM数据
result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
i = 0
for sec in selected_sec_ls:
    print(i, end='\r')
    ann_date = \
        df_filtered.loc[
            (df_filtered['code'] == sec) & (df_filtered['period'] == target_period), 'ann_date'].iloc[0]
    if len(period_list) > 1:
        indicator_data = \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                0] + \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[1]), indicator].iloc[
                0] - \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[2]), indicator].iloc[
                0]
    else:
        indicator_data = \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                0]
    result_df.loc[i] = [sec, ann_date, target_period, indicator_data]
    i += 1

In [129]:
# 从本地数据库获取财务指标数据
def get_TTM_fin_indicator_data_from_local(indicator='每股营业收入', target_period='20220331', threshold_date='20220520'):
    period_list = security_fundamental_data.get_TTM_period_list(target_period)
    df = security_fundamental_data.get_local_db_stock_fin_abstract(period=target_period, indicator=indicator)
    df = df.loc[df['ann_date'] <= threshold_date].copy()
    selected_sec_ls = df.code.tolist()
    if len(period_list) > 1:
        for period in period_list[1:]:
            new_df = security_fundamental_data.get_local_db_stock_fin_abstract(period=period, indicator=indicator)
            new_df = new_df.loc[new_df['ann_date'] <= threshold_date].copy()
            selected_sec_ls = list(set(selected_sec_ls) & set(new_df.code.tolist()))
            df = pd.concat([df, new_df], ignore_index=True)
    df_filtered = df.loc[df['code'].isin(selected_sec_ls)]  # 确保数据完整获取
    # 得到指定指标的TTM数据
    result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
    i = 0
    for sec in selected_sec_ls:
        # print(i, end='\r')
        ann_date = \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == target_period), 'ann_date'].iloc[0]
        if len(period_list) > 1:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                    0] + \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[1]), indicator].iloc[
                    0] - \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[2]), indicator].iloc[
                    0]
        else:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                    0]
        result_df.loc[i] = [sec, ann_date, target_period, indicator_data]
        i += 1
    return result_df


indicator = '每股营业收入'
target_period = '20220331'
threshold_date = '20220520'
get_TTM_fin_indicator_data_from_local(target_period=target_period, threshold_date=threshold_date, indicator=indicator)

,code,ann_date,period,每股营业收入_TTM
0,002350,20220428,20220331,4.223469
1,002413,20220428,20220331,1.291662
2,002572,20220428,20220331,11.667766
3,688560,20220427,20220331,8.528716
4,601212,20220429,20220331,10.755090
...,...,...,...,...
4194,600876,20220430,20220331,5.619953
4195,600057,20220428,20220331,225.725071
4196,600189,20220430,20220331,2.078305
4197,600362,20220429,20220331,133.676506


In [17]:
# 若该因子涉及多年数据，生成相关报告期的列表
def get_n_year_factor_period_ls(target_period, n=5):
    period_ls = []
    for i in range(n):
        new_period = str(int(target_period[:4]) - i) + target_period[-4:]
        period_ls.append(new_period)
    return period_ls

In [21]:
target_period = '20220331'
n = 5
indicator = indicator
threshold_date = '20220523'

period_list = get_n_year_factor_period_ls(target_period=target_period, n=n)
df = get_TTM_fin_indicator_data_from_local(target_period=target_period, indicator=indicator)
df = df.loc[df['ann_date'] <= threshold_date].copy()
selected_sec_ls = df.code.tolist()
for period in period_list[1:]:
    print(period, end='\r')
    new_df = get_TTM_fin_indicator_data_from_local(target_period=period, indicator=indicator)
    new_df = new_df.loc[new_df['ann_date'] <= threshold_date].copy()
    selected_sec_ls = list(set(selected_sec_ls) & set(new_df.code.tolist()))
    df = pd.concat([df, new_df], ignore_index=True)

In [39]:
df_filtered = df.loc[df['code'].isin(selected_sec_ls)]  # 确保数据完整获取
df_filtered = df_filtered.sort_values(by='period')
df_filtered = df_filtered.reset_index(inplace=False)
df_filtered

,index,code,ann_date,period,每股营业收入_TTM
0,17926,000068,20180417,20180331,0.428322
1,15882,600470,20180428,20180331,8.766777
2,15881,002688,20180423,20180331,2.293929
3,15880,000056,20180421,20180331,0.755863
4,15879,002482,20180424,20180331,8.467337
...,...,...,...,...,...
15285,2808,002231,20220429,20220331,1.411829
15286,2806,002288,20220427,20220331,2.679634
15287,2804,000999,20220423,20220331,15.996472
15288,2816,002793,20220430,20220331,4.725080


In [66]:
# 对每一只证券的5年每股净利润做线性回归，输出斜率数据
slope_df = pd.DataFrame(columns=['code', 'slope'])
sec = selected_sec_ls[0]
x = np.array([x + 1 for x in list(range(5))])
i = 0
for sec in selected_sec_ls:
    revenue_ls = df_filtered.loc[df_filtered['code'] == sec, '每股营业收入_TTM'].tolist()
    revenue_ps_mean = np.mean(revenue_ls)
    altered_revenue_ls = [x / revenue_ps_mean for x in revenue_ls]
    y = np.array(altered_revenue_ls)
    slope, intercept = np.polyfit(x, y, 1)
    slope_df.loc[i] = [sec, slope]
    i += 1

In [79]:
# 取因子排序后的前10%的证券，此处由大到小排序
threshold = slope_df['slope'].quantile(0.9)
filtered_df = slope_df[slope_df['slope'] >= threshold]
filtered_sec_list = filtered_df.code.tolist()

In [104]:
# 再根据盈利、流通市值筛选
df = pro.daily_basic(ts_code='', trade_date=threshold_date, fields='ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,circ_mv')
df['code'] = [x[:6] for x in df.ts_code.tolist()]
df = df.loc[df['code'].isin(filtered_sec_list)].copy()
df = df.loc[df['pe']>0]
df_sorted = df.sort_values(by='circ_mv',ignore_index=True)
df_sorted

In [128]:
sg_ls = df_sorted.code.tolist()

#2. 5年盈利增长率

In [130]:
# 从本地数据库获取财务指标数据
def get_TTM_fin_indicator_data_from_local(indicator='每股营业收入', target_period='20220331', threshold_date='20220520'):
    period_list = security_fundamental_data.get_TTM_period_list(target_period)
    df = security_fundamental_data.get_local_db_stock_fin_abstract(period=target_period, indicator=indicator)
    df = df.loc[df['ann_date'] <= threshold_date].copy()
    selected_sec_ls = df.code.tolist()
    if len(period_list) > 1:
        for period in period_list[1:]:
            new_df = security_fundamental_data.get_local_db_stock_fin_abstract(period=period, indicator=indicator)
            new_df = new_df.loc[new_df['ann_date'] <= threshold_date].copy()
            selected_sec_ls = list(set(selected_sec_ls) & set(new_df.code.tolist()))
            df = pd.concat([df, new_df], ignore_index=True)
    df_filtered = df.loc[df['code'].isin(selected_sec_ls)]  # 确保数据完整获取
    # 得到指定指标的TTM数据
    result_df = pd.DataFrame(columns=['code', 'ann_date', 'period', indicator + '_TTM'])
    i = 0
    for sec in selected_sec_ls:
        # print(i, end='\r')
        ann_date = \
            df_filtered.loc[
                (df_filtered['code'] == sec) & (df_filtered['period'] == target_period), 'ann_date'].iloc[0]
        if len(period_list) > 1:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                    0] + \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[1]), indicator].iloc[
                    0] - \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[2]), indicator].iloc[
                    0]
        else:
            indicator_data = \
                df_filtered.loc[
                    (df_filtered['code'] == sec) & (df_filtered['period'] == period_list[0]), indicator].iloc[
                    0]
        result_df.loc[i] = [sec, ann_date, target_period, indicator_data]
        i += 1
    return result_df


indicator = '基本每股收益'
target_period = '20220331'
threshold_date = '20220520'
get_TTM_fin_indicator_data_from_local(target_period=target_period, threshold_date=threshold_date, indicator=indicator)

,code,ann_date,period,基本每股收益_TTM
0,002350,20220428,20220331,0.1066
1,002413,20220428,20220331,-0.1900
2,002572,20220428,20220331,0.1270
3,688560,20220427,20220331,0.8000
4,601212,20220429,20220331,0.0162
...,...,...,...,...
4194,600876,20220430,20220331,0.3400
4195,600057,20220428,20220331,0.9860
4196,600189,20220430,20220331,0.1500
4197,600362,20220429,20220331,1.8100


In [131]:
target_period = '20220331'
n = 5
threshold_date = '20220523'

period_list = get_n_year_factor_period_ls(target_period=target_period, n=n)
df = get_TTM_fin_indicator_data_from_local(target_period=target_period, indicator=indicator)
df = df.loc[df['ann_date'] <= threshold_date].copy()
selected_sec_ls = df.code.tolist()
for period in period_list[1:]:
    print(period, end='\r')
    new_df = get_TTM_fin_indicator_data_from_local(target_period=period, indicator=indicator)
    new_df = new_df.loc[new_df['ann_date'] <= threshold_date].copy()
    selected_sec_ls = list(set(selected_sec_ls) & set(new_df.code.tolist()))
    df = pd.concat([df, new_df], ignore_index=True)

In [ ]:
df_filtered = df.loc[df['code'].isin(selected_sec_ls)]  # 确保数据完整获取
df_filtered = df_filtered.sort_values(by='period')
df_filtered = df_filtered.reset_index(inplace=False)
df_filtered

In [128]:
# 取因子排序后的前10%的证券，此处由大到小排序
threshold = slope_df['slope'].quantile(0.9)
filtered_df = slope_df[slope_df['slope'] >= threshold]
filtered_sec_list = filtered_df.code.tolist()

In [128]:
# 再根据盈利、流通市值筛选
df = pro.daily_basic(ts_code='', trade_date=threshold_date, fields='ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,circ_mv')
df['code'] = [x[:6] for x in df.ts_code.tolist()]
df = df.loc[df['code'].isin(filtered_sec_list)].copy()
df = df.loc[df['pe']>0]
df_sorted = df.sort_values(by='circ_mv',ignore_index=True)
df_sorted

In [128]:
eg_ls = df_sorted.code.tolist()